In [1]:
import os
import numpy as np
import pandas as pd
from math import acos, degrees
from itertools import combinations
from numpy.linalg import norm
from scipy.spatial import cKDTree

# --- Function Definitions ---
def parse_txt(filepath):
    vertices = []
    with open(filepath, 'r') as f:
        for line in f:
            parts = list(map(float, line.strip().split(',')))  # Assuming comma-separated values
            vertices.append(parts)
    return np.array(vertices)

def compute_features(vertices, k_neighbors=3):
    tree = cKDTree(vertices)
    features = []

    for i, j in combinations(range(len(vertices)), 2):
        p1, p2 = vertices[i], vertices[j]
        vec = p2 - p1
        dist = norm(vec)
        midpoint = (p1 + p2) / 2
        midpoint_height = midpoint[2]
        elevation_diff = abs(p1[2] - p2[2])

        horizontal = vec.copy(); horizontal[2] = 0
        angle_with_horizontal = degrees(acos(
            np.clip(np.dot(vec, horizontal) / (norm(vec) * norm(horizontal) + 1e-6), -1, 1)
        ))

        density_i = len(tree.query_ball_point(p1, r=2.0))
        density_j = len(tree.query_ball_point(p2, r=2.0))
        local_density = (density_i + density_j) / 2

        def get_planarity_score(point):
            _, idx = tree.query(point, k=k_neighbors)
            neighbor_pts = vertices[idx]
            cov = np.cov(neighbor_pts.T)
            eigvals = np.linalg.eigvalsh(cov)
            return eigvals[0] / (eigvals.sum() + 1e-6)

        planarity = (get_planarity_score(p1) + get_planarity_score(p2)) / 2

        def get_normal(point):
            _, idx = tree.query(point, k=k_neighbors)
            neighbor_pts = vertices[idx]
            cov = np.cov(neighbor_pts.T)
            eigvals, eigvecs = np.linalg.eigh(cov)
            return eigvecs[:, np.argmin(eigvals)]

        n1 = get_normal(p1)
        n2 = get_normal(p2)
        normal_diff = degrees(acos(np.clip(np.dot(n1, n2), -1, 1)))

        angles = []
        for k in range(len(vertices)):
            if k != j:
                vec2 = vertices[k] - p1
                angle = degrees(acos(
                    np.clip(np.dot(vec, vec2) / (norm(vec) * norm(vec2) + 1e-6), -1, 1)
                ))
                angles.append(angle)
        angle_to_others = np.mean(angles) if angles else 0

        features.append({
            'v1': i, 'v2': j,
            'distance': dist,
            'midpoint_height': midpoint_height,
            'angle_with_horizontal': angle_with_horizontal,
            'elevation_difference': elevation_diff,
            'local_density': local_density,
            'planarity_score': planarity,
            'normal_difference': normal_diff,
            'angle_to_other_edges': angle_to_others
        })

    return features

# === Set Path to TXT File ===
txt_file = r"C:\Users\akhil\OneDrive\Documents\3D wireframe reconstruction\RoofVE-main\pred_res\trivandrum\cluster_144_finCor_Geo.txt"

# --- Feature Extraction ---
try:
    verts = parse_txt(txt_file)
    print(f"Number of vertices: {len(verts)}")  # Check number of vertices parsed
    feats = compute_features(verts)
    print(f"Number of feature pairs: {len(feats)}")  # Check number of feature pairs computed
    for f in feats:
        f["file"] = os.path.basename(txt_file)
    all_features = feats
except Exception as e:
    print(f"Error processing {txt_file}: {e}")
    all_features = []

# --- Check if Features Exist and Save ---
df = pd.DataFrame(all_features)
print(f"Number of rows in the dataframe: {len(df)}")  # Check if dataframe has data
if not df.empty:
    df.to_csv("raw_vertex_pair_features_test.csv", index=False)
    print("Feature extraction complete. CSV file saved.")
else:
    print("No features to save.")


Number of vertices: 34
Number of feature pairs: 561
Number of rows in the dataframe: 561
Feature extraction complete. CSV file saved.


In [2]:
import pandas as pd

# Step 1: Load the dataset
df = pd.read_csv(r'raw_vertex_pair_features_test.csv')

# Step 2: Define features to drop
columns_to_drop = ['midpoint_height', 'planarity_score']

# Drop only the columns that actually exist in the DataFrame
columns_to_drop = [col for col in columns_to_drop if col in df.columns]
df_updated = df.drop(columns=columns_to_drop)

# Step 3: Save the updated dataset
df_updated.to_csv("important_final_test.csv", index=False)

print("✅ Updated dataset saved as 'important_final_test")

✅ Updated dataset saved as 'important_final_test


In [6]:
import pandas as pd
import numpy as np
import open3d as o3d
import joblib

# === Step 1: Load the Trained Model ===
model_path = r"C:\Users\akhil\OneDrive\Documents\3D wireframe reconstruction\saved_models_final\ExtraTrees_best_model.pkl"
model = joblib.load(model_path)

# === Step 2: Load Test Data ===
test_csv_path = r"important_final_test.csv"
df_test = pd.read_csv(test_csv_path)

# === Step 3: Load the saved Scaler ===
scaler_path = r"C:\Users\akhil\OneDrive\Documents\3D wireframe reconstruction\saved_models_final\standard_scaler.pkl"
scaler = joblib.load(scaler_path)

# === Step 4: Prepare and Scale Test Features ===
feature_columns = [
    'distance', 'angle_with_horizontal',
    'elevation_difference', 'local_density',
    'normal_difference', 'angle_to_other_edges'
]
X_test = df_test[feature_columns].values
X_test_scaled = scaler.transform(X_test)

# === Step 5: Predict Probabilities ===
y_pred_probs = model.predict_proba(X_test_scaled)
df_test['probability'] = y_pred_probs[:, 1]

# === Step 6: Dynamic Thresholding Based on Top-K% Confident Edges ===
top_k_percent = 0.20 #Keep top 55% confident edges
threshold_dynamic = df_test['probability'].quantile(1 - top_k_percent)
df_test['prediction'] = (df_test['probability'] >= threshold_dynamic).astype(int)

# === Step 7: Extract Predicted Edges ===
# Handle NaN values in 'v1' and 'v2'
df_test = df_test.dropna(subset=['v1', 'v2'])

# Now you can safely convert 'v1' and 'v2' to integers
predicted_edges = [
    (int(row['v1']), int(row['v2']))
    for idx, row in df_test.iterrows() if row['prediction'] == 1
]

#print("✅ Number of predicted edges after dynamic thresholding:", len(predicted_edges))

# === Step 8: Save Predictions and Probabilities to CSV ===
predictions_csv_path = r"C:\Users\akhil\OneDrive\Documents\3D wireframe reconstruction\predictions_with_probabilities.csv"
df_test[['v1', 'v2', 'prediction', 'probability']].to_csv(predictions_csv_path, index=False)
print(f"✅ Predictions and probabilities saved to: {predictions_csv_path}")

# === Step 9: Load TXT vertices ===
def parse_txt_vertices(filepath):
    vertices = []
    with open(filepath, 'r') as f:
        for line in f:
            parts = list(map(float, line.strip().split(',')))  # Assuming comma-separated values
            vertices.append(parts)
    return np.array(vertices)

# Update the file path for your .txt file
txt_file_path = r"C:\Users\akhil\OneDrive\Documents\3D wireframe reconstruction\RoofVE-main\pred_res\trivandrum\cluster_144_finCor_Geo.txt"
verts_np = parse_txt_vertices(txt_file_path)

# === Step 10: Visualize with Open3D ===
# Create point cloud from vertices
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(verts_np)
pcd.paint_uniform_color([0.5, 0.5, 0.5])  # Gray points

# Create LineSet with black edges
line_set = o3d.geometry.LineSet()
line_set.points = o3d.utility.Vector3dVector(verts_np)
line_set.lines = o3d.utility.Vector2iVector(predicted_edges)
line_set.paint_uniform_color([0, 0, 0])  # Black edges

# Visualize
print("🚀 Visualizing predicted edges and vertices with black color...")
o3d.visualization.draw_geometries([pcd, line_set], window_name="Predicted Edges with Black Coloring")


C:\Users\akhil\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


✅ Predictions and probabilities saved to: C:\Users\akhil\OneDrive\Documents\3D wireframe reconstruction\predictions_with_probabilities.csv
🚀 Visualizing predicted edges and vertices with black color...
